# Toronto Dwellings Analysis Dashboard By: Ebad Salehi

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [4]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [5]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)
type(map_box_api)

str

# Import Data

In [6]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path, index_col="neighbourhood")
#to_data.head(-10)

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data

In [7]:
## Dwelling Types Per Year

# Used for "number_dwelling_types"
df_dwelling_units = to_data.reset_index()
df_dwelling_units = to_data[["single_detached_house","apartment_five_storeys_plus",
                             "movable_dwelling","semi_detached_house","row_house","duplex",
                             "apartment_five_storeys_less","other_house"]].groupby("year").sum()
#df_dwelling_units.head()

In [8]:
## Calculate the "average monthly shelter costs per year" for owned and rented dwellings

# Used for "average_house_value" and for "create_line_chart(fn)"
avg_month_dwellings = to_data[["shelter_costs_owned","shelter_costs_rented"]].groupby("year").mean()
#avg_month_dwellings

# Used for "Average House Value per Year"
avg_house_val = to_data["average_house_value"].groupby("year").mean()

# Plot the average house value per year as a line chart Using create_line_chart fn
#avg_house_bar = create_line_chart(avg_house_val,"Average House Value in Toronto", "year", "Avg House Value", "blue")


In [9]:
# Used for average_house_value by Neighbourhood
avg_neighbourhood_val = to_data.reset_index()
avg_neighbourhood_val = avg_neighbourhood_val[["year","neighbourhood","average_house_value"]]

In [10]:
#Used for Number of Dwelling Types per Year
dwells_number = to_data.reset_index()
dwells_number = dwells_number.drop(columns=["average_house_value","shelter_costs_owned","shelter_costs_rented"], axis=1)
#dwells_number.head(10)

In [11]:
# Used for Top 10 Most Expensive Neighbourhoods
df_top_10= to_data.groupby("neighbourhood").mean()
df_top_10 = df_top_10.sort_values(by="average_house_value", ascending=False).head(10)
df_top_10.reset_index(inplace=True)
#df_top_10

plot_df_top10 = df_top_10.set_index("neighbourhood")
plot_df_top10 = plot_df_top10["average_house_value"]
#plot_df_top10

In [12]:
# Used for neighbourhood_map

#df_neighbourhood_locations.head()

#using base dataframe (to_data), groupby "neighbourhood" and calculated the mean
neighbourhood_df = to_data.reset_index()
neighbourhood_df = neighbourhood_df.drop("year", axis=1)
df_neighbourhood_mean = neighbourhood_df.groupby("neighbourhood").mean()
df_neighbourhood_mean.head(-10)

# Join the average values with the neighbourhood locations
df_joined_neighbourhood= pd.concat([df_neighbourhood_locations, df_neighbourhood_mean], axis="columns", join="outer")
#preventing the concat TO togenerate NaN
df_neighbourhood_locations.reset_index(drop=True, inplace=True)
df_neighbourhood_mean.reset_index(drop=True, inplace=True)
#df_joined_neighbourhood.head(-5)

# Create a scatter mapbox to analyze neighbourhood info
# Using df_joined_neighbourhood DataFrame
map_plot = px.scatter_mapbox(
    df_joined_neighbourhood,
    lat="lat",
    lon="lon",
    color="average_house_value",
    title ="Average House Values in Toronto",
    mapbox_style="open-street-map")
#map_plot.show()

In [13]:
# Used for average_house_value_snapshot
reset_to_data = to_data.reset_index()
row_data = reset_to_data.rename({"average_house_value":"Avg. House Value"}, axis=1)
#row_data

In [14]:
# Creating bar chart "row facet" for plot using the reset to_data
df_facet = to_data.reset_index()
dwellings_plot_row = px.bar(
    df_facet,
    x="neighbourhood",
    y="average_house_value",
    color="average_house_value",
    facet_row="year",
    title="Number of Dwelling Types Per Year",
    width=1200,
    height=1200)

In [15]:
# Used for sunburts_cost_analysis
df_top_sorted = to_data.sort_values(by=["year","average_house_value"], ascending=(False,False))
df_top_sorted.reset_index(inplace=True)
#df_top_sorted.head(-5)

In [16]:
# Used for sunburts_cost_analysis
options = [2001]
top_10_2001 = df_top_sorted.loc[df_top_sorted["year"].isin(options)]
top_10_2001.sort_values(by="average_house_value", ascending=False).head(10)
top_10_2001 = top_10_2001.iloc[0:10]
#top_10_2001

In [17]:
# Used for sunburts_cost_analysis
options = [2006]
top_10_2006 = df_top_sorted.loc[df_top_sorted["year"].isin(options)]
top_10_2006.sort_values(by="average_house_value", ascending=False).head(10)
top_10_2006 = top_10_2006.iloc[0:10]
#top_10_2006

In [18]:
# Used for sunburts_cost_analysis
options = [2011]
top_10_2011 = df_top_sorted.loc[df_top_sorted["year"].isin(options)]
top_10_2011.sort_values(by="average_house_value", ascending=False).head(10)
top_10_2011 = top_10_2011.iloc[0:10]
#top_10_2011

In [19]:
# Used for sunburts_cost_analysis
options = [2016]
top_10_2016 = df_top_sorted.loc[df_top_sorted["year"].isin(options)]
top_10_2016.sort_values(by="average_house_value", ascending=False).head(10)
top_10_2016 = top_10_2016.iloc[0:10]
#top_10_2016

In [20]:
# Used for sunburts_cost_analysis
df_top_10_combined = pd.concat([top_10_2001,top_10_2006,top_10_2011,top_10_2016])
#df_top_10_combined.head()

In [21]:
# Create the sunburst chart using "df_top_10_combined"
sunburst_chart =px.sunburst(
    df_top_10_combined,
    path=['year','neighbourhood'],
    values="average_house_value",
    color='shelter_costs_owned',
    title="Cost per Year of Most Expensive Neighbourhoods in Toronto",
    height=700
)
#sunburst_chart.show()

### Panel Visualization Functions

In [22]:
# Define Panel visualization functions

"""This fn is for 1st(welcome) tab"""

def neighbourhood_map():
    """Neighbourhood Map"""
    map_plot = px.scatter_mapbox(
    neighbourhood_joined_df,
    lat="lat",
    lon="lon",
    color="average_house_value",
    title ="Average House Values in Toronto",
    mapbox_style="open-street-map")
    map_plot.show()
    return(map_plot)

In [23]:
"""This fn is for 2nd tab"""

def create_bar_chart(data, title, xlabel, ylabel, color):
    create_bar_chart = data.hvplot.bar(title=title, xlabel=xlabel, ylabel=ylabel, color=color, rot=90, height=400)
    return (create_bar_chart)

dwell_2001_bar = create_bar_chart(df_dwelling_units.iloc[0],"Dwelling Types Toronto 2001", "2001","Dwelling Type Units", "red").opts(yformatter="%.0f")
dwell_2006_bar = create_bar_chart(df_dwelling_units.iloc[1],"Dwelling Types Toronto 2006", "2006","Dwelling Type Units", "royalblue").opts(yformatter="%.0f")
dwell_2011_bar = create_bar_chart(df_dwelling_units.iloc[2],"Dwelling Types Toronto 2011", "2011","Dwelling Type Units", "orange").opts(yformatter="%.0f")
dwell_2016_bar = create_bar_chart(df_dwelling_units.iloc[3],"Dwelling Types Toronto 2016", "2016","Dwelling Type Units", "pink").opts(yformatter="%.0f")


In [24]:
"""This fn is for 3rd tab"""

def create_line_chart(data, title, xlabel, ylabel, color):
    create_line_chart = data.hvplot.line(title=title, xlabel=xlabel, ylabel=ylabel, color=color, height=400)
    return (create_line_chart)

owned_dwellings_line= create_line_chart(avg_month_dwellings["shelter_costs_owned"],"Average Monthly Shelter Cost for Owned Dwellings in Toronto", "year", "Avg Monthly Shelter Costs", "blue")
rented_dwellings_line = create_line_chart(avg_month_dwellings["shelter_costs_rented"],"Average Monthly Shelter Cost for Rented Dwellings in Toronto", "year", "Avg Monthly Shelter Costs", "red")


In [25]:
"""This fn is for 3rd tab"""

def average_house_value (data, title, xlabel, ylabel, color):
    """Average house values per year."""
    average_line_chart = data.hvplot.line(x=x, y=y, groupby=groupby, height=400)    
    return(average_line_chart)

avg_house_bar = create_line_chart(avg_house_val,"Average House Value in Toronto", "year", "Avg House Value", "blue")


In [26]:
"""This fn is for 4th tab"""

def average_value_by_neighbourhood(data, title, xlabel, ylabel, color):
    """Average house values by neighbourhood."""
    average_value_line_chart = data.hvplot.line(x=x, y=y, groupby=groupby, height=400)    
    return(average_value_line_chart)

average_value_line_plot = avg_house_val.hvplot.line(x="year", y="average_house_value").opts(yformatter="%.0f")


In [27]:
"""This fn is for 4th tab"""

def number_dwelling_types(data, title, xlabel, ylabel, color):
    """Number of dwelling types per year"""
    number_dwelling_line_chart = data.hvplot.line(x=x, y=y, groupby=groupby, height=400)    
    return(number_dwelling_line_chart)

num_dwell_type_plot = dwells_number.hvplot.bar(x="year", groupby="neighbourhood", rot=90).opts(yformatter="%.0f")


In [28]:
"""This fn is for 4th tab"""

def average_house_value_snapshot():
    """Average house value for all Toronto's neighbourhoods per year."""
    row_dwellings_plot = px.bar(
    row_data,
    x="neighbourhood",
    y="Avg. House Value",
    color="Avg. House Value",
    facet_row="year",
    title="Number of Dwelling Types Per Year",
    width=1200,
    height=1200)

    return(row_dwellings_plot)


In [29]:
"""This fn is for 5th tab"""

def top_most_expensive_neighbourhoods():
    """Top 10 most expensive neighbourhoods."""
    top_10_line_chart = data.hvplot.line(x=x, y=y, groupby=groupby, height=400)    
    return(top_10_line_chart)
    
top_10_plot = plot_df_top10.hvplot.bar(x="neighbourhood", ylabel="Avg. House Value", title="The Top 10 Most Expensive Neighbourhoods", rot=90, height=600).opts(yformatter="%.0f")


In [30]:
"""This fn is for 5th tab"""

def sunburts_cost_analysis():
    """Sunburst chart to conduct a costs analysis of most expensive neighbourhoods in Toronto per year."""
    
    combo_sunburst =px.sunburst(
    top_10_sun_combo,
    path=['year','neighbourhood'],
    values="average_house_value",
    color='shelter_costs_owned',
    title="Costs of Most Expensive Neighbourhoods in Toronto per Year",
    height=700)
    return(combo_sunburst)

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [31]:
# Create a Title for the Dashboard
dasboard_title= "Toronto's Real Estate Market Analysis (2001 to 2016)"

# Define a welcome text
dashboard_desc = "This dashboard provides Maps, Interactive Visualizations, and Static Charts to help the investors analyze the property market of Toronto!"

# Creating tabs layout for the dashboard
start_here_tab = pn.Column("This dashboard provides Maps, Interactive Visualizations, and Static Charts to help the investors analyze the property market of Toronto!", map_plot)

bar_charts_tab = pn.Column(
    "Annual Market Oversight Based on 'Dwelling Types' ", dwell_2016_bar, dwell_2011_bar, dwell_2006_bar, dwell_2001_bar)

shelters_tab = pn.Column(
    "Buy or Rent ? Based on 'Average Housing Price' ", owned_dwellings_line, rented_dwellings_line, avg_house_bar)

top_10_expensive_neighb = pn.Row(top_10_plot , sunburst_chart)

neighbourhoods_tab = pn.Column(
    "Neighbourhood Analysis", average_value_line_plot, num_dwell_type_plot, dwellings_plot_row)


In [32]:
# Create the main dashboard
dashboard = pn.Tabs(
    ("Start Here!", start_here_tab),
    ("Annual Market Oversight", bar_charts_tab ),
    ("Shelter Costs VS House Value", shelters_tab),
    ("Top 10 Expensive Suburbs", top_10_expensive_neighb),
    ("Neighbourhood Analysis", neighbourhoods_tab)    
)

## Serve the Panel Dashboard

In [34]:
dashboard.servable()
#dashboard.show()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
    [1] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
        [2] HoloViews(Bars)
        [3] HoloViews(Bars)
        [4] HoloViews(Bars)
    [2] Column
        [0] Markdown(str)
        [1] HoloViews(Curve)
        [2] HoloViews(Curve)
        [3] HoloViews(Curve)
    [3] Row
        [0] HoloViews(Bars)
        [1] Plotly(Figure)
    [4] Column
        [0] Markdown(str)
        [1] HoloViews(Curve)
        [2] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North', width=250)
                [1] VSpacer()
        [3] Plotly(Figure, viewport={'xaxis.range': [-0.5, ...})